# Vehicle and Lane Detection

In [1]:
import cv2
import pickle
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage.measurements import label
from collections import deque
from utils.featureExtraction import *

## 1. Setting Lane Detection Pipeline

In [2]:
# Camera Calibration
import os
from utils import cameraCalibration
(ret, cameraMat, distCoeffs, rvecs, tvecs), fig = cameraCalibration.get_calibration_matrix(os.path.join('camera_cal','*.jpg'))
plt.close()

# Perspective Transform Params
img_size = (1280, 720)
width, height = img_size
offset = 200
src = np.float32([
    [  588,   446 ],
    [  691,   446 ],
    [ 1126,   673 ],
    [  153 ,   673 ]])
dst = np.float32([[offset, 0], [img_size[0] - offset, 0], [img_size[0] - offset, img_size[1]], [offset, img_size[1]]])
M = cv2.getPerspectiveTransform(src,dst)
Minv = cv2.getPerspectiveTransform(dst, src)

# Lane Detection Pipeline
from utils.lane_pipeline import LanePipeline
from utils.line import Line
line=Line()
LanePipeline.set_values(line, M, Minv, cameraMat, distCoeffs)

## 2. Setting Vehicle Detection Pipeline

In [3]:
from utils.vehicle_detector import VehicleDetector
vehicleDetector = VehicleDetector('model-params.pk')
vehicleDetector.ystart_ystop_scale = [(380, 480, 1), (400, 600, 1.5), (500, 700, 2.5)]
vehicleDetector.threshold = 3

## 3. Combining Lane and Vehicle Detection

In [4]:
def process_image(img):
    lane_detected = LanePipeline.pipeline(img)
    return vehicleDetector.find_cars(lane_detected)

In [5]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

white_output = 'project_video_output.mp4'
clip = VideoFileClip("project_video.mp4")#.subclip(t_start=30,t_end=35)
white_clip = clip.fl_image(process_image)
%time white_clip.write_videofile(white_output, audio=False)

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

[MoviePy] >>>> Building video project_video_output.mp4
[MoviePy] Writing video project_video_output.mp4


100%|█████████████████████████████████████████████████████████████████████████████▉| 1260/1261 [11:34<00:00,  1.50it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_video_output.mp4 

Wall time: 11min 36s
